In [5]:
import cv2
from tqdm import tqdm
print(cv2.__version__)

4.8.1


In [6]:
a= tqdm()

0it [00:00, ?it/s]

2: : 0it [01:19, ?it/s]

In [3]:
vidcap = cv2.VideoCapture("Videos/Start1.avi")
print (vidcap.read())
success,image = vidcap.read()
count = 0
success = True
while success:
    a.set_description(f'{count}')
    success,image = vidcap.read()
    cv2.imwrite("Images/test1/frame%d.jpg" % count, image)     # save frame as JPEG file
    count += 1

(True, array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8))
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True


KeyboardInterrupt: 